### Set up the Cassandra CQL Context.  Set the default keyspace

In [5]:
import org.apache.spark.sql.cassandra.CassandraSQLContext
val csc = new CassandraSQLContext(sc)
csc setKeyspace "music"

In [16]:
%%cql create table music.tracks_by_year (dummy text, year int, track_count int, primary key (dummy, track_count) ) with clustering order by (track_count desc)

### Create a dataframe based on an SQL statement

In [40]:
val tmp = csc.sql("select 'dummy' as dummy, album_year as year, count(*) as track_count from tracks_by_album group by album_year")

In [41]:
tmp.show

+-----+----+-----------+
|dummy|year|track_count|
+-----+----+-----------+
|dummy|1952|         10|
|dummy|1956|         53|
|dummy|1957|         21|
|dummy|1958|         31|
|dummy|1959|         28|
|dummy|1960|         42|
|dummy|1961|         86|
|dummy|1962|        129|
|dummy|1963|        101|
|dummy|1964|        147|
|dummy|1965|        203|
|dummy|1966|         95|
|dummy|1967|        244|
|dummy|1968|        243|
|dummy|1969|        326|
|dummy|1970|        379|
|dummy|1971|        420|
|dummy|1972|        262|
|dummy|1973|        443|
|dummy|1974|        489|
+-----+----+-----------+



#### Register the dataframe as a temp table

In [43]:
tmp registerTempTable "tmp_tracks_by_year"

In [44]:
csc.sql("insert into table music.tracks_by_year select dummy, track_count, year from tmp_tracks_by_year")

org.apache.spark.sql.DataFrame = [dummy: string, track_count: int, year: int]

In [46]:
%%cql select * from music.tracks_by_year limit 10

dummy,track_count,year
dummy,2200,1998
dummy,2170,2001
dummy,2028,1999
dummy,1984,2002
dummy,1980,2000
dummy,1708,1995
dummy,1607,1996
dummy,1604,1997
dummy,1544,1994
dummy,1353,1993


### Or ... the one-liner technique. Lets save the data with a simple insert - select statement

In [48]:
%%cql truncate music.tracks_by_year

In [51]:
csc.sql("insert into table music.tracks_by_year select 'dummy' as dummy, album_year, count(*) as track_count from music.tracks_by_album group by album_year")

org.apache.spark.sql.DataFrame = [dummy: string, track_count: int, year: int]

In [52]:
%%cql select * from music.tracks_by_year limit 10

dummy,track_count,year
dummy,2009,22
dummy,2008,11
dummy,2007,77
dummy,2006,229
dummy,2005,688
dummy,2004,1004
dummy,2003,1331
dummy,2002,1984
dummy,2001,2170
dummy,2000,1980
